# Лабораторная работа №1: Исследование алгоритма k-ближайших соседей (KNN)

## 1. Выбор начальных условий

**Набор данных для классификации**

В качестве набора данных для задачи классификации был выбран **Banknote Authentication Data Set**.

**Практическая задача:** Разработка автоматизированной системы для определения подлинности банкнот критически важна для банковской сферы, ритейла и любых систем, принимающих наличные, для предотвращения финансовых потерь от фальшивых денег.

Ссылка на данные: [https://archive.ics.uci.edu/ml/datasets/banknote+authentication](https://archive.ics.uci.edu/ml/datasets/banknote+authentication)

In [15]:
import kagglehub
import pandas as pd

# Скачиваем самую актуальную версию
path = kagglehub.dataset_download("ritesaluja/bank-note-authentication-uci-data")

# Загружаем данные для классификации
df_clf = pd.read_csv(path + "/BankNote_Authentication.csv")

# Выведем первые 5 строк, чтобы убедиться, что загрузка прошла успешно
print("Данные для классификации:")
print(df_clf.head())

Данные для классификации:
   variance  skewness  curtosis  entropy  class
0   3.62160    8.6661   -2.8073 -0.44699      0
1   4.54590    8.1674   -2.4586 -1.46210      0
2   3.86600   -2.6383    1.9242  0.10645      0
3   3.45660    9.5228   -4.0112 -3.59440      0
4   0.32924   -4.4552    4.5718 -0.98880      0


**Набор данных для регрессии**

В качестве набора данных для задачи регрессии был выбран **Medical Cost Personal Datasets**.

**Практическая задача:** Прогнозирование индивидуальных медицинских расходов позволяет страховым компаниям справедливо рассчитывать стоимость страховых полисов, а государству - планировать бюджет на здравоохранение.


Ссылка на данные: [https://www.kaggle.com/datasets/mirichoi0218/insurance](https://www.kaggle.com/datasets/mirichoi0218/insurance)

In [17]:
import kagglehub

# Скачиваем самую актуальную версию
path = kagglehub.dataset_download("mirichoi0218/insurance")

# Загружаем данные для регрессии
df_reg = pd.read_csv(path + "/insurance.csv")

# Выведем первые 5 строк, чтобы убедиться, что загрузка прошла успешно
print("Данные для регрессии:")
print(df_reg.head())

Данные для регрессии:
   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


**Выбор метрик качества**

**Для задачи классификации:**
1.  **Accuracy (доля правильных ответов)** - процент объектов, верно классифицированных моделью. Простая и в то же время интуитивная метрика.
2.  **F1-score** - среднее гармоническое между точностью (precision) и полнотой (recall). Более продвинутая метрика. Особенно полезна, если классы несбалансированы.

**Для задачи регрессии:**
1.  **Mean Absolute Error (MAE, средняя абсолютная ошибка)** - показывает, насколько в среднем модель ошибается в прогнозах. Легко интерпретируется.
2.  **R² (коэффициент детерминации)** - показывает, какую долю дисперсии (разброса) целевой переменной объясняет наша модель.. Чем ближе к 1, тем лучше.

## 2. Создание бейзлайна и оценка качества

**Подготовка данных**

In [18]:
# ЗАДАЧА КЛАССИФИКАЦИИ

# Отделяем признаки (X) от целевой переменной (y)
X_clf = df_clf.drop('class', axis=1)
y_clf = df_clf['class']

# Разделяем данные на обучающую и тестовую выборки
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42)

# ЗАДАЧА РЕГРЕССИИ

# В данных регрессии есть категориальные признаки (sex, smoker, region).
# Простые модели не умеют с ними работать. Для бейзлайна мы их удалим.
df_reg_baseline = df_reg.drop(['sex', 'smoker', 'region'], axis=1)

# Отделяем признаки (X) от целевой переменной (y)
X_reg = df_reg_baseline.drop('charges', axis=1)
y_reg = df_reg_baseline['charges']

# Разделяем данные на обучающую и тестовую выборки
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42)

**Обучение модели**

In [19]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error, r2_score

# ЗАДАЧА КЛАССИФИКАЦИИ
# Создаем модель. k=5 - это распространенное стартовое значение
knn_clf = KNeighborsClassifier(n_neighbors=5)
# Обучаем модель на обучающих данных
knn_clf.fit(X_clf_train, y_clf_train)

# ЗАДАЧА РЕГРЕССИИ
knn_reg = KNeighborsRegressor(n_neighbors=5)
knn_reg.fit(X_reg_train, y_reg_train)

KNeighborsRegressor()

**Делаем предсказания и считаем метрики**

In [20]:
# ЗАДАЧА КЛАССИФИКАЦИИ
# Делаем предсказания на тестовых данных
y_clf_pred = knn_clf.predict(X_clf_test)

# Считаем метрики
acc_baseline = accuracy_score(y_clf_test, y_clf_pred)
f1_baseline = f1_score(y_clf_test, y_clf_pred)

print("Классификация:")
print(f"Accuracy: {acc_baseline:.4f}")
print(f"F1-score: {f1_baseline:.4f}")

# ЗАДАЧА РЕГРЕССИИ
# Делаем предсказания на тестовых данных
y_reg_pred = knn_reg.predict(X_reg_test)

# Считаем метрики
mae_baseline = mean_absolute_error(y_reg_test, y_reg_pred)
r2_baseline = r2_score(y_reg_test, y_reg_pred)

print("\nРегрессия")
print(f"Mean Absolute Error: {mae_baseline:.2f}")
print(f"R²: {r2_baseline:.4f}")

Классификация:
Accuracy: 1.0000
F1-score: 1.0000

Регрессия
Mean Absolute Error: 9017.63
R²: -0.0091


Мы получили для классификации высокие результаты, а для регрессии - очень плохие.

## 3. Улучшение бейзлайна

**Гипотезы для улучшения**

1.  **Гипотеза 1 (Общая):** Алгоритм KNN основан на расстоянии. Если один признак измеряется в тысячах, а другой - в десятках, то первый будет иметь гораздо большее влияние на расстояние. Масштабирование признаков должно улучшить качество предсказаний.
2.  **Гипотеза 2 (Общая):** `k=5` был выбран наугад. Возможно, другое количество соседей сработает лучше. Нужно подобрать оптимальное `k`.
3.  **Гипотеза 3 (Для регрессии):** В бейзлайне мы удалили важные категориальные признаки (`sex`, `smoker`, `region`). Если мы их закодируем, модель сможет их использовать, и качество должно вырасти.

Проверим их.

In [21]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# ЗАДАЧА КЛАССИФИКАЦИИ (Гипотезы 1 и 2)

# 1. Создаем пайплайн
pipeline_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# 2. Задаем сетку параметров для подбора k
param_grid_clf = {'knn__n_neighbors': range(1, 11)}

# 3. Используем GridSearchCV для подбора лучшего k с кросс-валидацией
grid_search_clf = GridSearchCV(pipeline_clf, param_grid_clf, cv=5, scoring='f1')
grid_search_clf.fit(X_clf_train, y_clf_train)

# Выводим лучший параметр k
print(f"Лучшее k для классификации: {grid_search_clf.best_params_['knn__n_neighbors']}")
# Улучшенная модель
improved_model_clf = grid_search_clf

Лучшее k для классификации: 1


`k = 1` объясняется тем, что классы в наборе данных Banknote Authentication очень хорошо отделяются друг от друга в пространстве признаков.

In [23]:
# ЗАДАЧА РЕГРЕССИИ (Гипотезы 1, 2, 3)

# 1. На этот раз не удаляем категориальные признаки
X_reg_full = df_reg.drop('charges', axis=1)
y_reg_full = df_reg['charges']

X_reg_full_train, X_reg_full_test, y_reg_full_train, y_reg_full_test = train_test_split(
    X_reg_full, y_reg_full, test_size=0.2, random_state=42)

# 2. Определяем, какие колонки числовые, а какие категориальные
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['sex', 'smoker', 'region']

# 3. Создаем препроцессор
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features), # Для числовых - масштабирование
        ('cat', OneHotEncoder(), categorical_features)]) # Для категориальных - One-Hot Encoding

# 4. Создаем пайплайн
pipeline_reg = Pipeline([
    ('preprocessor', preprocessor),
    ('knn', KNeighborsRegressor())
])

# 5. Сетка параметров для подбора
param_grid_reg = {'knn__n_neighbors': range(1, 25)}

# 6. Подбираем лучшее k
grid_search_reg = GridSearchCV(pipeline_reg, param_grid_reg, cv=5, scoring='r2')
grid_search_reg.fit(X_reg_full_train, y_reg_full_train)

print(f"Лучшее k для регрессии: {grid_search_reg.best_params_['knn__n_neighbors']}")
improved_model_reg = grid_search_reg

Лучшее k для регрессии: 6


**Делаем предсказания на улучшенных моделях и считаем метрики**

In [24]:
# Классификация
y_clf_pred_imp = improved_model_clf.predict(X_clf_test)
acc_improved = accuracy_score(y_clf_test, y_clf_pred_imp)
f1_improved = f1_score(y_clf_test, y_clf_pred_imp)

# Регрессия
y_reg_pred_imp = improved_model_reg.predict(X_reg_full_test)
mae_improved = mean_absolute_error(y_reg_full_test, y_reg_pred_imp)
r2_improved = r2_score(y_reg_full_test, y_reg_pred_imp)

# СРАВНЕНИЕ РЕЗУЛЬТАТОВ

print("Сравнение результатов")
print("Классификация:")
print(f"Accuracy: {acc_baseline:.4f} -> {acc_improved:.4f}")
print(f"F1-score: {f1_baseline:.4f} -> {f1_improved:.4f}")
print("\nРегрессия:")
print(f"MAE: {mae_baseline:.2f} -> {mae_improved:.2f}")
print(f"R²: {r2_baseline:.4f} -> {r2_improved:.4f}")

Сравнение результатов
Классификация:
Accuracy: 1.0000 -> 1.0000
F1-score: 1.0000 -> 1.0000

Регрессия:
MAE: 9017.63 -> 3680.04
R²: -0.0091 -> 0.7565


**Выводы по улучшению**

**Классификация:**
- Изначальный бейзлайн уже показывал крайне высокий результат, что уже говорит о хорошей разделимости данных.
- После масштабирования признаков и подбора оптимального `k` качество предсказаний не изменилось.

**Регрессия:**
- Бейзлайн был крайне слабым, так как не использовал категориальные признаки и не масштабировал числовые.
- После применения пайплайна с масштабированием числовых признаков, кодированием категориальных и подбором оптимального `k`, качество модели выросло. R² стал положительным и довольно высоким, а MAE значительно снизилась.


Это доказывает, что предобработка данных — критически важный этап, который может превратить бесполезную модель в эффективную. Все гипотезы подтвердились.

## 4.	Имплементация алгоритма машинного обучения

In [25]:
import numpy as np
from collections import Counter

# Функция для расчета евклидова расстояния
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))

class MyKNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        # Просто запоминает данные
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        # Делаем предсказания для каждого объекта в X_test
        predictions = [self._predict(x) for x in X_test]
        return np.array(predictions)

    def _predict(self, x):
        # 1. Рассчитать расстояния от x до всех точек в self.X_train
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # 2. Найти k ближайших соседей
        # Получаем индексы k точек с наименьшими расстояниями
        k_indices = np.argsort(distances)[:self.k]

        # 3. Получаем метки (классы или значения) этих соседей
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # 4. Сделать предсказание
        if isinstance(self.y_train[0], (int, str, bool)): # Если метки - целые числа или строки -> классификация
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        else: # Иначе -> регрессия
            return np.mean(k_nearest_labels)


X_clf_train_np = X_clf_train.to_numpy()
y_clf_train_np = y_clf_train.to_numpy()
X_clf_test_np = X_clf_test.to_numpy()

X_reg_train_np = X_reg_train.to_numpy()
y_reg_train_np = y_reg_train.to_numpy()
X_reg_test_np = X_reg_test.to_numpy()

In [26]:
# КЛАССИФИКАЦИЯ
my_knn_clf = MyKNN(k=5)
my_knn_clf.fit(X_clf_train_np, y_clf_train_np)
my_y_clf_pred = my_knn_clf.predict(X_clf_test_np)

my_acc = accuracy_score(y_clf_test, my_y_clf_pred)
my_f1 = f1_score(y_clf_test, my_y_clf_pred)

print("Своя реализация vs Sklearn (Бейзлайн)")
print("Классификация:")
print(f"Accuracy: {acc_baseline:.4f} (sklearn) -> {my_acc:.4f} (своя)")
print(f"F1-score: {f1_baseline:.4f} (sklearn) -> {my_f1:.4f} (своя)")


# РЕГРЕССИЯ
my_knn_reg = MyKNN(k=5)
my_knn_reg.fit(X_reg_train_np, y_reg_train_np)
my_y_reg_pred = my_knn_reg.predict(X_reg_test_np)

my_mae = mean_absolute_error(y_reg_test, my_y_reg_pred)
my_r2 = r2_score(y_reg_test, my_y_reg_pred)

print("\nРегрессия")
print(f"MAE: {mae_baseline:.2f} (sklearn) -> {my_mae:.2f} (своя)")
print(f"R²: {r2_baseline:.4f} (sklearn) -> {my_r2:.4f} (своя)")

Своя реализация vs Sklearn (Бейзлайн)
Классификация:
Accuracy: 1.0000 (sklearn) -> 1.0000 (своя)
F1-score: 1.0000 (sklearn) -> 1.0000 (своя)

Регрессия
MAE: 9017.63 (sklearn) -> 9028.61 (своя)
R²: -0.0091 (sklearn) -> -0.0101 (своя)


**Выводы по своей реализации (сравнение с бейзлайном)**

Результаты моей реализации KNN с `k=5` на безлайне практически полностью совпадают с результатами `sklearn.neighbors.KNeighbors...` с `k=5`. Это подтверждает, что базовая логика алгоритма реализована корректно. Небольшие расхождения могут быть связаны с внутренними оптимизациями или способами обработки одинаковых расстояний в `sklearn`.

In [28]:
# КЛАССИФИКАЦИЯ (с улучшениями)

# 1. Получаем лучшее k, найденное ранее
best_k_clf = grid_search_clf.best_params_['knn__n_neighbors']

# 2. Получаем доступ к шагу 'scaler' из нашего пайплайна
scaler_clf = grid_search_clf.best_estimator_.named_steps['scaler']

# 3. Масштабируем данные
X_clf_train_scaled = scaler_clf.transform(X_clf_train)
X_clf_test_scaled = scaler_clf.transform(X_clf_test)

# 4. Обучаем нашу модель на масштабированных данных
my_knn_clf_imp = MyKNN(k=best_k_clf)
my_knn_clf_imp.fit(X_clf_train_scaled, y_clf_train_np)
my_y_clf_pred_imp = my_knn_clf_imp.predict(X_clf_test_scaled)

# 5. Оценка
my_acc_imp = accuracy_score(y_clf_test, my_y_clf_pred_imp)
my_f1_imp = f1_score(y_clf_test, my_y_clf_pred_imp)


# РЕГРЕССИЯ (с улучшениями)

# 1. Получаем лучшее k, найденное ранее
best_k_reg = grid_search_reg.best_params_['knn__n_neighbors']

# 2. Получаем доступ к препроцессору
preprocessor_reg = grid_search_reg.best_estimator_.named_steps['preprocessor']

# 3. Преобразуем данные
X_reg_train_processed = preprocessor_reg.transform(X_reg_full_train)
X_reg_test_processed = preprocessor_reg.transform(X_reg_full_test)

# 4. Обучаем нашу модель
my_knn_reg_imp = MyKNN(k=best_k_reg)
my_knn_reg_imp.fit(X_reg_train_processed, y_reg_full_train.to_numpy())
my_y_reg_pred_imp = my_knn_reg_imp.predict(X_reg_test_processed)

# 5. Оценка
my_mae_imp = mean_absolute_error(y_reg_full_test, my_y_reg_pred_imp)
my_r2_imp = r2_score(y_reg_full_test, my_y_reg_pred_imp)


# ИТОГОВОЕ СРАВНЕНИЕ

print("Своя реализация vs Sklearn (Улучшенные)")
print("Классификация:")
print(f"Accuracy: {acc_improved:.4f} (sklearn) -> {my_acc_imp:.4f} (своя)")
print(f"F1-score: {f1_improved:.4f} (sklearn) -> {my_f1_imp:.4f} (своя)")
print("\nРегрессия:")
print(f"MAE: {mae_improved:.2f} (sklearn) -> {my_mae_imp:.2f} (своя)")
print(f"R²: {r2_improved:.4f} (sklearn) -> {my_r2_imp:.4f} (своя)")

Своя реализация vs Sklearn (Улучшенные)
Классификация:
Accuracy: 1.0000 (sklearn) -> 1.0000 (своя)
F1-score: 1.0000 (sklearn) -> 1.0000 (своя)

Регрессия:
MAE: 3680.04 (sklearn) -> 3680.04 (своя)
R²: 0.7565 (sklearn) -> 0.7565 (своя)


### Выводы по своей реализации (сравнение с улучшенной моделью)

После применения тех же техник предобработки (масштабирование, кодирование) и использования оптимального `k`, найденного через GridSearchCV, моя реализация алгоритма KNN показала результаты, идентичные улучшенной модели из `sklearn`.

**Общий вывод по лабораторной работе №1:**
1.  Алгоритм KNN является мощным, но чувствительным к масштабу признаков и выбору гиперпараметра `k`.
2.  Правильная предобработка данных (масштабирование, работа с категориями) является необходимой для получения качественных результатов, особенно для метрических алгоритмов.
3.  Собственная реализация алгоритма позволила глубже понять его внутреннюю работу и подтвердила, что его базовая логика достаточно проста, а высокая производительность `sklearn` достигается за счет оптимизированных вычислений и продуманной архитектуры (Pipeline, GridSearchCV).